In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tree import AFTSurvivalTree
from forest import AFTForest
from dataset import SupportDataset, SyntheticDataset, VeteranLungDataset
import time

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('../data/veterans_lung_cancer.csv')
df.describe()

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
count,137.000000,137.0,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000
mean,121.627737,inf,58.306569,58.569343,8.773723,0.197080,0.197080,0.350365,0.255474,0.708029,0.291971,0.503650,0.496350
std,157.816722,NaN,10.541628,20.039592,10.612141,0.399253,0.399253,0.478835,0.437728,0.456337,0.456337,0.501821,0.501821
min,1.000000,1.0,34.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,25.0,51.000000,40.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,80.0,62.000000,60.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,144.000000,177.0,66.000000,75.000000,11.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,999.000000,inf,81.000000,99.000000,87.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
df

,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
0,72.0,72.0,69.0,60.0,7.0,0,0,0,1,1,0,1,0
1,411.0,411.0,64.0,70.0,5.0,0,0,0,1,0,1,1,0
2,228.0,228.0,38.0,60.0,3.0,0,0,0,1,1,0,1,0
3,126.0,126.0,63.0,60.0,9.0,0,0,0,1,0,1,1,0
4,118.0,118.0,65.0,70.0,11.0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,133.0,133.0,65.0,75.0,1.0,0,1,0,0,1,0,0,1
133,111.0,111.0,64.0,60.0,5.0,0,1,0,0,1,0,0,1
134,231.0,231.0,67.0,70.0,18.0,0,1,0,0,0,1,0,1
135,378.0,378.0,65.0,80.0,4.0,0,1,0,0,1,0,0,1


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, genextreme, gaussian_kde, ecdf

In [5]:
veteran = VeteranLungDataset(df)
veteran.get_data()

array([[69., 60.,  7., ...,  0.,  1.,  0.],
       [64., 70.,  5., ...,  1.,  1.,  0.],
       [38., 60.,  3., ...,  0.,  1.,  0.],
       ...,
       [67., 70., 18., ...,  1.,  0.,  1.],
       [65., 80.,  4., ...,  0.,  0.,  1.],
       [37., 30.,  3., ...,  0.,  0.,  1.]])

In [6]:
veteran.get_label()

array([(0,  72.), (0, 411.), (0, 228.), (0, 126.), (0, 118.), (0,  10.),
       (0,  82.), (0, 110.), (0, 314.), (1, 100.), (0,  42.), (0,   8.),
       (0, 144.), (1,  25.), (0,  11.), (0,  30.), (0, 384.), (0,   4.),
       (0,  54.), (0,  13.), (1, 123.), (1,  97.), (0, 153.), (0,  59.),
       (0, 117.), (0,  16.), (0, 151.), (0,  22.), (0,  56.), (0,  21.),
       (0,  18.), (0, 139.), (0,  20.), (0,  31.), (0,  52.), (0, 287.),
       (0,  18.), (0,  51.), (0, 122.), (0,  27.), (0,  54.), (0,   7.),
       (0,  63.), (0, 392.), (0,  10.), (0,   8.), (0,  92.), (0,  35.),
       (0, 117.), (0, 132.), (0,  12.), (0, 162.), (0,   3.), (0,  95.),
       (0, 177.), (0, 162.), (0, 216.), (0, 553.), (0, 278.), (0,  12.),
       (0, 260.), (0, 200.), (0, 156.), (1, 182.), (0, 143.), (0, 105.),
       (0, 103.), (0, 250.), (0, 100.), (0, 999.), (0, 112.), (1,  87.),
       (1, 231.), (0, 242.), (0, 991.), (0, 111.), (0,   1.), (0, 587.),
       (0, 389.), (0,  33.), (0,  25.), (0, 357.), 

In [7]:
veteran.get_xgboost_label()

,Survival_label_lower_bound,Survival_label_upper_bound
0,72.0,72.0
1,411.0,411.0
2,228.0,228.0
3,126.0,126.0
4,118.0,118.0
...,...,...
132,133.0,133.0
133,111.0,111.0
134,231.0,231.0
135,378.0,378.0


In [8]:
X_train, X_test, y_train, y_test = veteran.get_train_test()

##### Init All Methods

AFT-Forest

In [9]:
aft_surv_tree = AFTSurvivalTree(function="norm")


In [10]:
 kwargs = {
    "is_bootstrap": False, 
    "is_custom_dist": False, 
    "n_components": 1000,
    "max_depth": 10,
    "min_samples_split": 2, 
    "min_samples_leaf": 2,
    "sigma": 0.1,
    "n_samples": 1000,
    "percent_len_sample": 0.5,
    "test_size": 0.5
}
aft_forest = AFTForest(n_trees=10, random_state=42, **kwargs)

In [11]:
aft_surv_tree.fit(X_train, y_train)

In [12]:
aft_forest.fit(X_train, y_train)

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/m

In [13]:
aft_surv_tree.predict(X_test)

[125.5,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 70.2,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 70.2,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 104.1,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 104.1,
 127.28915662650603]

In [14]:
aft_forest.predict(X_test)

[110.5437468671679,
 114.84566628108732,
 100.87608733371891,
 114.84566628108732,
 117.56004723346828,
 106.98795951417003,
 120.79143776106932,
 120.87899961442065,
 111.01408733371889,
 111.01408733371889,
 105.79995951417001,
 108.22195199537302,
 128.9146833751044,
 111.79146828609984,
 107.69513495276651,
 115.07480161943317,
 120.87899961442065,
 111.01408733371889,
 107.69513495276651,
 122.42066628108732,
 115.07480161943317,
 111.01408733371889,
 112.7391520467836,
 100.87608733371891,
 117.56004723346828,
 120.79143776106932,
 115.07480161943317,
 111.01408733371889]

In [15]:
aft_surv_tree._score(X_test, y_test)

0.5206043956043956

In [16]:
aft_surv_tree._visualize()

In [17]:
aft_surv_tree._brier(X_test, y_test)

0.10341047841047839

In [18]:
aft_surv_tree._mae(X_test, y_test)

99.91157487091223

In [19]:
aft_surv_tree._auc(X_test, y_test)

(array([0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.49487179, 0.5       , 0.50274725,
        0.5085034 , 0.46071429, 0.47152194, 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5 

In [20]:
aft_forest._score(X_test, y_test)

0.5535714285714286

In [21]:
aft_forest._brier(X_test, y_test)

0.09919247419247418

In [22]:
aft_forest._auc(X_test, y_test)

(array([0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5085034 , 0.434375  , 0.38888889, 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5 

In [23]:
aft_surv_tree_weibull = AFTSurvivalTree(function="weibull", is_bootstrap=False, is_custom_dist=True)


In [24]:
aft_surv_tree_weibull.fit(X_train, y_train)

In [25]:

aft_surv_tree_weibull._score(X_test, y_test)

0.5

In [26]:
aft_surv_tree_weibull.save("tree_weibull.json")

In [27]:
aft_surv_tree_load_test = AFTSurvivalTree()
aft_surv_tree_load_test = aft_surv_tree_load_test.load("tree_weibull.json")

In [28]:
aft_surv_tree_load_test._score(X_test, y_test)

0.5

In [29]:
aft_surv_tree_logistic = AFTSurvivalTree(function="logistic", is_bootstrap=False, is_custom_dist=True)

In [30]:
aft_surv_tree_logistic.fit(X_train, y_train)

In [31]:
aft_surv_tree_logistic._score(X_test, y_test)

0.5

In [32]:
aft_surv_tree_gmm = AFTSurvivalTree(function="gmm", is_bootstrap=False, is_custom_dist=True)

In [33]:
aft_surv_tree_gmm.fit(X_train, y_train)

In [34]:
aft_surv_tree_gmm._score(X_test, y_test)

0.5

In [35]:
aft_surv_tree_normal = AFTSurvivalTree(function="norm", is_bootstrap=False, is_custom_dist=True)

In [36]:
aft_surv_tree_normal.fit(X_train, y_train)


In [37]:
aft_surv_tree_normal._score(X_test, y_test)

0.5

In [38]:
aft_surv_tree_extreme = AFTSurvivalTree(function="extreme", is_bootstrap=True, is_custom_dist=True)

In [39]:
aft_surv_tree_extreme.fit(X_train, y_train)

In [40]:
aft_surv_tree_extreme._score(X_test, y_test)

0.5

In [41]:
aft_surv_tree_extreme._visualize()

In [42]:
aft_surv_tree_extreme.save("extreme.json")

In [43]:
kwargs = {
    "function": "gmm", 
    "is_bootstrap": True, 
    "is_custom_dist": True,
    "n_components": 2
}
aft_forest = AFTForest(n_trees=10, **kwargs)

In [44]:
aft_forest.fit(X_train, y_train)

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
100%|██████████| 10/10 [00:00<00:00, 82402.83it/s]


In [45]:
aft_forest._score(X_test, y_test)

0.6167582417582418

In [46]:
aft_forest.save("forest.json")

'models/forest/forest.json'

In [47]:
# aft_forest_load_test._score(X_test, y_test)

In [48]:
# aft_forest_load_test.save("forest_load.json")

In [49]:
kwargs = {
    "function": "norm", 
    "is_bootstrap": False, 
    "is_custom_dist": True
}
aft_forest_1 = AFTForest(n_trees=10, **kwargs)

In [50]:
aft_forest_1.fit(X_train, y_train)

/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/distribution.py:444: RuntimeWarning: overflow encountered in exp
  return lognorm.pdf(x, self.sigma_, loc=0, scale=np.exp(self.mu_))
/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/distribution.py:450: RuntimeWarning: overflow encountered in exp
  return lognorm.cdf(x, self.sigma_, loc=0, scale=np.exp(self.mu_))
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2071: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/distribution.py:450: RuntimeWarning: overflow encountered in exp
  return lognorm.cdf(x, self.sigma_, loc=0, scale=np.exp(self.mu_))
/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/distribution.py:444: RuntimeWarning: overflow encountered in exp
  return lognorm.pdf(x, self.sigma_, loc=0, scale=np.ex

In [51]:
aft_forest_1._score(X_test, y_test)

0.4725274725274725

In [52]:
aft_forest_1._mae(X_test, y_test)

97.88150895601981

In [53]:
kwargs = {
    "function": "norm",
    "is_bootstrap": False, 
    "is_custom_dist": False,
    "n_components": 3
}
aft_forest_2 = AFTForest(n_trees=10, **kwargs)

In [54]:
aft_forest_2.fit(X_train, y_train)

100%|██████████| 10/10 [00:00<00:00, 73843.38it/s]


In [55]:

aft_forest_2._score(X_test, y_test)

0.6181318681318682

In [56]:
aft_forest_2._mae(X_test, y_test)

95.99825863479052

In [57]:
from runner import run_n_models, tune_model

# c_indexes, briers, maes = run_n_models(AFTForest, X_train, y_train, X_test, y_test, n_models=5)

In [58]:
# res = tune_model(AFTForest, X_train, y_train, X_test, y_test, is_cv=True, n_tries=5, n_models=5, n_splits=5)

In [59]:
# res

In [60]:
y_pred = aft_forest_2.predict(X_test)

In [61]:
y_test

array([(0,  51.), (0,  80.), (0, 144.), (0, 151.), (0,  45.), (0, 103.),
       (0, 143.), (0,   8.), (0, 111.), (0,  30.), (0,   1.), (0,   8.),
       (0, 999.), (0, 250.), (0, 139.), (0,  63.), (0,  22.), (0,  13.),
       (0, 133.), (0, 216.), (0,  52.), (0, 118.), (1,  83.), (0,  18.),
       (0,  25.), (0,  99.), (0, 384.), (0,  54.)],
      dtype=(numpy.record, [('death', '<i8'), ('d.time', '<f8')]))

In [62]:
y_true = [time for _, time in y_test]
y_true = np.array(y_true)
y_true

array([ 51.,  80., 144., 151.,  45., 103., 143.,   8., 111.,  30.,   1.,
         8., 999., 250., 139.,  63.,  22.,  13., 133., 216.,  52., 118.,
        83.,  18.,  25.,  99., 384.,  54.])

In [63]:
censor_info = [censor for censor, _ in y_test]
censor_info = np.array(censored)
censor_info

NameError: name 'censored' is not defined

In [ ]:
y_pred = np.array(y_pred)

In [ ]:
y_pred, y_true

In [ ]:
from utils import plot_survival_trees
plot_survival_trees(y_true, y_pred, censor_info)